In [243]:
import sys
import pandas as pd
import os
from keplergl import KeplerGl

import pickle
from h3 import h3

In [244]:
df_roads = pd.read_csv("./data/train/roads_dataset.csv")
df_transport = pd.read_csv("data/train/routes_dataset.csv")
df_population = pd.read_csv("data/train/rosstat_population_all_cities.csv")
df_isochrones = pd.read_csv("data/train/isochrones_walk_dataset.csv")
df_stops = pd.read_csv("data/train/osm_stops.csv")
df_companies = pd.read_csv("data/train/osm_amenity.csv")
df_target = pd.read_csv("data/train/target_hakaton_spb.csv", encoding = "Windows-1251", sep = ";").drop_duplicates()

In [245]:
df_target

,gorod,geo_h3_10,lat_h3,lon_h3,atm_category,target,atm_cnt
0,Екатеринбург,8a10c2283357fff,56.865861,60.532224,category4,95.333333,1
1,Екатеринбург,8a10c2288087fff,56.885391,60.588203,category3,30652.666667,4
2,Екатеринбург,8a10c228820ffff,56.891648,60.605399,category2,1162.166667,1
3,Екатеринбург,8a10c2288adffff,56.895039,60.598777,category1,2961.333333,1
4,Екатеринбург,8a10c228944ffff,56.896206,60.625911,category1,2963.333333,1
...,...,...,...,...,...,...,...
533,Санкт-Петербург,8a1106ab67a7fff,59.983488,30.227538,category1,2535.833333,1
534,Санкт-Петербург,8a1106ab6807fff,59.990519,30.205250,category1,23519.500000,2
535,Санкт-Петербург,8a1106ab6807fff,59.990519,30.205250,category4,94.500000,1
536,Санкт-Петербург,8a1106ab6ae7fff,59.996443,30.219656,category2,489.500000,1


In [246]:
df_target_n = df_target[['geo_h3_10','target','atm_cnt','atm_category','lat_h3','lon_h3']]

In [247]:
df_target_n

,geo_h3_10,target,atm_cnt,atm_category,lat_h3,lon_h3
0,8a10c2283357fff,95.333333,1,category4,56.865861,60.532224
1,8a10c2288087fff,30652.666667,4,category3,56.885391,60.588203
2,8a10c228820ffff,1162.166667,1,category2,56.891648,60.605399
3,8a10c2288adffff,2961.333333,1,category1,56.895039,60.598777
4,8a10c228944ffff,2963.333333,1,category1,56.896206,60.625911
...,...,...,...,...,...,...
533,8a1106ab67a7fff,2535.833333,1,category1,59.983488,30.227538
534,8a1106ab6807fff,23519.500000,2,category1,59.990519,30.205250
535,8a1106ab6807fff,94.500000,1,category4,59.990519,30.205250
536,8a1106ab6ae7fff,489.500000,1,category2,59.996443,30.219656


In [235]:
##df_target_n = df_target.drop(columns = ['gorod'], inplace = True)

In [248]:
df_target_n = df_target_n.set_index('atm_category')

In [249]:
n = int(input("Введите номер категории(от 1 до 4): "))
nmax = 4
for i in range(1,5):
    if (i % (nmax+1)) != n:
        df_target_n.drop(['category'+str(i)], axis = 0,inplace = True)

Введите номер категории(от 1 до 4): 4


In [250]:
df_target_n

,geo_h3_10,target,atm_cnt,lat_h3,lon_h3
atm_category,,,,,
category4,8a10c2283357fff,95.333333,1,56.865861,60.532224
category4,8a10c2289517fff,116.350000,1,56.893980,60.612610
category4,8a10c2289737fff,84.750000,1,56.897425,60.626473
category4,8a10c22898f7fff,88.333333,1,56.906647,60.614352
category4,8a10c2289a1ffff,154.333333,1,56.910091,60.628219
...,...,...,...,...,...
category4,8a1106ab3987fff,103.833333,1,60.007397,30.259688
category4,8a1106ab47b7fff,108.666667,1,60.000349,30.209289
category4,8a1106ab5727fff,87.500000,1,60.022805,30.225546


In [251]:
df_target_n = df_target_n.reset_index()

In [252]:
df_target_n = df_target_n.eval('avg= target / atm_cnt')

In [253]:
df_target_n

,atm_category,geo_h3_10,target,atm_cnt,lat_h3,lon_h3,avg
0,category4,8a10c2283357fff,95.333333,1,56.865861,60.532224,95.333333
1,category4,8a10c2289517fff,116.350000,1,56.893980,60.612610,116.350000
2,category4,8a10c2289737fff,84.750000,1,56.897425,60.626473,84.750000
3,category4,8a10c22898f7fff,88.333333,1,56.906647,60.614352,88.333333
4,category4,8a10c2289a1ffff,154.333333,1,56.910091,60.628219,154.333333
...,...,...,...,...,...,...,...
145,category4,8a1106ab3987fff,103.833333,1,60.007397,30.259688,103.833333
146,category4,8a1106ab47b7fff,108.666667,1,60.000349,30.209289,108.666667
147,category4,8a1106ab5727fff,87.500000,1,60.022805,30.225546,87.500000
148,category4,8a1106ab6807fff,94.500000,1,59.990519,30.205250,94.500000


In [254]:
df_target_avg = df_target_n[['geo_h3_10', 'avg']]

In [255]:
df_pop = df_population[['geo_h3_10', 'population', 'lat', 'lon']]
df_target_avg_pop = df_target_avg.merge(df_pop, how = 'left', on = 'geo_h3_10')

In [256]:
df_target_avg_pop

,geo_h3_10,avg,population,lat,lon
0,8a10c2283357fff,95.333333,NaN,NaN,NaN
1,8a10c2289517fff,116.350000,375.0,56.893980,60.612610
2,8a10c2289737fff,84.750000,654.0,56.897425,60.626473
3,8a10c22898f7fff,88.333333,NaN,NaN,NaN
4,8a10c2289a1ffff,154.333333,NaN,NaN,NaN
...,...,...,...,...,...
145,8a1106ab3987fff,103.833333,NaN,NaN,NaN
146,8a1106ab47b7fff,108.666667,NaN,NaN,NaN
147,8a1106ab5727fff,87.500000,NaN,NaN,NaN
148,8a1106ab6807fff,94.500000,NaN,NaN,NaN


In [257]:
df_target_end = df_target_avg_pop.merge(df_companies, how = 'left', on = 'geo_h3_10')

In [261]:
df_target_end

,geo_h3_10,avg,population,lat_x,lon_x,city,Автозапчасти для иномарок,Авторемонт и техобслуживание (СТО),Алкогольные напитки,Аптеки,...,Постаматы,Продуктовые магазины,Пункты выдачи интернет-заказов,Рестораны,Страхование,Супермаркеты,Цветы,Шиномонтаж,lat_y,lon_y
0,8a10c2283357fff,95.333333,NaN,NaN,NaN,Екатеринбург,NaN,NaN,NaN,2.0,...,NaN,NaN,1.0,NaN,1.0,6.0,3.0,NaN,56.865861,60.532224
1,8a10c2289517fff,116.350000,375.0,56.893980,60.612610,Екатеринбург,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,56.893980,60.612610
2,8a10c2289737fff,84.750000,654.0,56.897425,60.626473,Екатеринбург,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,56.897425,60.626473
3,8a10c22898f7fff,88.333333,NaN,NaN,NaN,Екатеринбург,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,4.0,NaN,NaN,56.906647,60.614352
4,8a10c2289a1ffff,154.333333,NaN,NaN,NaN,Екатеринбург,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,2.0,NaN,NaN,56.910091,60.628219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,8a1106ab3987fff,103.833333,NaN,NaN,NaN,Санкт-Петербург,NaN,NaN,1.0,NaN,...,1.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,60.007397,30.259688
146,8a1106ab47b7fff,108.666667,NaN,NaN,NaN,Санкт-Петербург,NaN,NaN,NaN,2.0,...,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,60.000349,30.209289
147,8a1106ab5727fff,87.500000,NaN,NaN,NaN,Санкт-Петербург,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,60.022805,30.225546
148,8a1106ab6807fff,94.500000,NaN,NaN,NaN,Санкт-Петербург,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,2.0,1.0,NaN,59.990519,30.205250


In [262]:
df_target_end = df_target_end.drop(columns = ['lat_x','lon_x','city'])

In [263]:
df_target_end.to_csv("some_data_end"+str(n)+".csv")